In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/edsa-movie-recommender-challenge-2022/sample_submission.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/imdb_data.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_scores.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/links.csv


In [2]:
train_df  = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv')
movies_df = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv')


In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_error
from math import sqrt
from surprise import SVD
import surprise
from surprise import Reader, Dataset
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances 


In [4]:
train_df.head()


,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [5]:
test_df.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14310 entries, 0 to 14309
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   userId   14310 non-null  int64
 1   movieId  14310 non-null  int64
dtypes: int64(2)
memory usage: 223.7 KB


In [7]:
submission_df = test_df.copy()
submission_df['Id'] = submission_df['userId'].map(str) +'_'+ submission_df['movieId'].map(str)
submission_df = submission_df.drop(['userId', 'movieId'], axis = 1)
submission_df

,Id
0,5_788
1,68_7438
2,336_40412
3,803_3822
4,547_903
...,...
14305,294_30707
14306,803_780
14307,519_912
14308,628_6764


In [8]:
train_df.drop(columns=['timestamp'],inplace=True)

In [9]:
train_df.head(10)

,userId,movieId,rating
0,5163,57669,4.0
1,106343,5,4.5
2,146790,5459,5.0
3,106362,32296,2.0
4,9041,366,3.0
5,120949,81768,3.0
6,19630,62049,4.0
7,21066,2282,1.0
8,117563,120474,4.0
9,144018,1997,5.0


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000038 entries, 0 to 10000037
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 228.9 MB


In [11]:
train_df.drop(train_df[train_df['userId'] >1000].index, inplace=True)
#sample = train_df.sample(n = 15000, random_state = 2021)

In [12]:
train_df

,userId,movieId,rating
735,469,410,3.0
815,216,1690,3.0
836,628,135569,2.5
882,321,1127,4.0
935,886,117533,4.5
...,...,...,...
9998866,693,165,4.0
9999005,871,5954,4.0
9999166,847,2018,4.0
9999295,859,6323,4.5


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57238 entries, 735 to 9999768
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   57238 non-null  int64  
 1   movieId  57238 non-null  int64  
 2   rating   57238 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.7 MB


In [14]:
train_df.sort_values('userId')

,userId,movieId,rating
718844,1,1217,3.5
9947467,1,5912,3.0
2072860,1,8685,1.0
2533005,1,665,5.0
2313219,1,7937,3.0
...,...,...,...
6998003,1000,1079,3.5
166592,1000,500,2.0
4438832,1000,3526,3.5
915819,1000,6936,0.5


In [15]:
train_pivot = train_df.pivot(index='userId',columns='movieId',values='rating')
train_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,203519,203649,204352,204692,204698,205106,205499,206272,206499,208002
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
train_pivot.fillna(0,inplace=True)

In [17]:
train_pivot.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,203519,203649,204352,204692,204698,205106,205499,206272,206499,208002
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
res = cosine_similarity(train_pivot)
res

array([[1.        , 0.        , 0.        , ..., 0.        , 0.01302579,
        0.06570509],
       [0.        , 1.        , 0.02770318, ..., 0.        , 0.05853344,
        0.15828523],
       [0.        , 0.02770318, 1.        , ..., 0.        , 0.04921286,
        0.03693188],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.02041514,
        0.02599856],
       [0.01302579, 0.05853344, 0.04921286, ..., 0.02041514, 1.        ,
        0.13871468],
       [0.06570509, 0.15828523, 0.03693188, ..., 0.02599856, 0.13871468,
        1.        ]])

In [19]:
similarity_table = pd.DataFrame(data = res, 
                               index = train_pivot.index,
                               columns = train_pivot.index)
                  
similarity_table

userId,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.028553,0.039611,0.000000,0.000000,0.038735,0.030097,0.000000,...,0.000000,0.000000,0.042659,0.000000,0.000000,0.000000,0.012362,0.000000,0.013026,0.065705
2,0.000000,1.000000,0.027703,0.059181,0.037379,0.000000,0.061299,0.028198,0.047674,0.000000,...,0.000000,0.026676,0.095005,0.072782,0.009326,0.002908,0.041902,0.000000,0.058533,0.158285
3,0.000000,0.027703,1.000000,0.155263,0.048103,0.048722,0.026178,0.044322,0.017327,0.080592,...,0.000000,0.011392,0.092098,0.000000,0.075390,0.075294,0.205464,0.000000,0.049213,0.036932
4,0.028553,0.059181,0.155263,1.000000,0.041190,0.037054,0.000000,0.086602,0.033253,0.000000,...,0.000000,0.018005,0.042586,0.000000,0.029975,0.027340,0.120978,0.000000,0.027161,0.050031
5,0.039611,0.037379,0.048103,0.041190,1.000000,0.038554,0.065597,0.120142,0.146536,0.179508,...,0.000000,0.160571,0.113236,0.059911,0.072475,0.000000,0.029511,0.000000,0.154636,0.098854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.000000,0.002908,0.075294,0.027340,0.000000,0.087487,0.000000,0.062388,0.017735,0.110703,...,0.000000,0.000000,0.018769,0.000000,0.113885,1.000000,0.049489,0.000000,0.053835,0.077893
997,0.012362,0.041902,0.205464,0.120978,0.029511,0.024065,0.021837,0.074908,0.043717,0.023853,...,0.023159,0.035000,0.094387,0.046656,0.080377,0.049489,1.000000,0.034937,0.082908,0.067351
998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051805,0.062894,0.000000,...,0.000000,0.049621,0.022821,0.055542,0.000000,0.000000,0.034937,1.000000,0.020415,0.025999


In [20]:
def movie_rating(reference_user, reference_movie):
    user_ref = similarity_table[reference_user]
    similiar_user = pd.DataFrame(user_ref)
    similiar_user = (similiar_user.sort_values(reference_user, ascending = False)).iloc[1:21]
    similiar_user.rename(columns={reference_user:'sim_to_current_user'},inplace=True)
    most_similiar_user_rating = train_pivot.filter([reference_movie])
    most_similiar_user_rating.rename(columns={reference_movie:'most_similiar_user_ratings_of_current_movie'},inplace=True)
    final_dataset = pd.merge(similiar_user, most_similiar_user_rating, on = 'userId')
    movie_rating = (final_dataset['sim_to_current_user'] * final_dataset['most_similiar_user_ratings_of_current_movie']).sum() / (final_dataset['sim_to_current_user']).sum()
    final_movie_rating = round(movie_rating*2)/2
    
    return final_movie_rating

In [21]:
movie_rating(803, 3822)

0.0

In [22]:
submission_df['rating'] = test_df.apply(lambda x: movie_rating(x['userId'], x['movieId']), axis=1)

In [23]:
submission_df.head(20)

,Id,rating
0,5_788,0.5
1,68_7438,1.0
2,336_40412,0.0
3,803_3822,0.0
4,547_903,0.5
5,514_5379,0.0
6,367_190939,0.0
7,345_3007,0.5
8,468_5782,0.5
9,441_2908,0.5


In [24]:
submission_df.to_csv(r'dconradie3.csv', index = False)